# Real-time Face Detection and Recognition

In [ ]:
!pip install tensorflow mtcnn opencv-python
import tensorflow as tf
from mtcnn import MTCNN
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
# Load the trained model and class names
model = tf.keras.models.load_model('face_recognition_model.h5')
class_names = np.load('face_classes.npy')
detector = MTCNN()

In [ ]:
def detect_and_recognize():
    def take_photo(filename='photo.jpg'):
        js = Javascript('''
            async function takePhoto() {
                const stream = await navigator.mediaDevices.getUserMedia({video: true});
                const video = document.createElement('video');
                const div = document.createElement('div');
                document.body.appendChild(div);
                div.appendChild(video);
                video.srcObject = stream;
                await video.play();

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                canvas.getContext('2d').drawImage(video, 0, 0);
                const img = canvas.toDataURL('image/jpeg');

                stream.getVideoTracks()[0].stop();
                div.remove();
                return img;
            }
            ''')
        display(js)
        data = eval_js('takePhoto()')
        binary = b64decode(data.split(',')[1])
        return binary

    # Capture and process frame
    binary = take_photo()
    nparr = np.frombuffer(binary, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Detect faces
    faces = detector.detect_faces(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    for face in faces:
        x, y, width, height = face['box']
        face_img = frame[y:y+height, x:x+width]
        face_img = cv2.resize(face_img, (160, 160))
        face_img = face_img / 255.0
        
        # Predict
        prediction = model.predict(np.expand_dims(face_img, axis=0))
        predicted_class = class_names[np.argmax(prediction)]
        confidence = np.max(prediction)

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, f"{predicted_class} ({confidence:.2f})",
                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (0, 255, 0), 2)

    cv2_imshow(frame)

# Run detection loop
while True:
    try:
        detect_and_recognize()
    except KeyboardInterrupt:
        break